In [ ]:
# !pip install scikit-surprise

In [1]:
import pandas as pd
from surprise import *

In [31]:
df1 = pd.read_excel('4eme.xlsx')
#df1 = df1.fillna(0.01)

df1.head(5)

,id,23,23.1,23.2,23.3,24,24.1,24.2,24.3,30,...,496.2,496.3,497,497.1,497.2,497.3,524,524.1,524.2,524.3
0,index,NaN,NaN,NaN,NaN,2,2,2,2,NaN,...,12,12,14,14,14,14,11,11,11,11
1,name,الحواس,الحواس,الحواس,الحواس,التنقل,التنقل,التنقل,التنقل,أكون الأعداد ذات 5 أرقام و أفككها و أركبها,...,Léo et Lina 2,Léo et Lina 2,La soupe aux légumes de Lina,La soupe aux légumes de Lina,La soupe aux légumes de Lina,La soupe aux légumes de Lina,Léo et Lina 1,Léo et Lina 1,Léo et Lina 1,Léo et Lina 1
2,subject,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,إيقاظ علمي,رياضيات,...,Français,Français,Français,Français,Français,Français,Français,Français,Français,Français
3,**,correct questions,num of mistakes,num of not Completed exercises,%,correct questions,num of mistakes,num of not Completed exercises,%,correct questions,...,num of not Completed exercises,%,correct questions,num of mistakes,num of not Completed exercises,%,correct questions,num of mistakes,num of not Completed exercises,%
4,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
# Merge subject/module names with feature names
new_cols = []
for i,j,k in zip(list(df1.iloc[3]), list(df1.iloc[2]),  list(df1.iloc[1])):
      new_cols.append(i + " :" + j + " (" + k + ")")

p_cols = []
#Keep only the percentage for each module
for col in new_cols:
   if "**" in col or "%" in col :
      p_cols.append(col)

# Create the new dataset
new_df = df1
new_df.columns = new_cols
new_df = new_df.drop([0,1,2,3],axis=0)
new_df = new_df[p_cols]
new_df

,** :subject (name),% :إيقاظ علمي (الحواس),% :إيقاظ علمي (التنقل),% :رياضيات (أكون الأعداد ذات 5 أرقام و أفككها و أركبها),"% :رياضيات (الأعداد ذات خمسة أرقام , مقارنة و ترتيب)",% :رياضيات (أتصرف في القطع النقدية و الأوراق المالية),"% :رياضيات (أتصرف في وحدات قيس الأطوال , المتر و أجزاؤه)","% :رياضيات (المسالك على الشبكة , اختصار المسالك و تعيين على موقع الشبكة)",% :رياضيات (تعرف المستقيم و أجزاءه),% :رياضيات (ألعاب تحدي درجة 1),...,% :Français (L'oiseau blessé 2),% :عربية (تونس الجميلة),% :عربية (تضامُنٌ),% :عربية (أريدُ هاتفًا جوّالاً),% :عربية (بائـــــــعُ الزُّهـــــــور),% :عربية (الأسد و الفأر),% :Français (Léo et Lina 3),% :Français (Léo et Lina 2),% :Français (La soupe aux légumes de Lina),% :Français (Léo et Lina 1)
4,55,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,71,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,114,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,119,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5591,71388,NaN,NaN,0.08,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5592,71405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5593,71409,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5594,71417,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
# Build training dataframe based on modules 
final_df = new_df.melt(id_vars="** :subject (name)")
# Trun % progress into a rating /10 
# final_df['value'] = final_df['value']*10 
# Rename columns 
final_df.columns = ["user", "item", "rating"]
# Remove the % from the modules names
final_df['item'] = final_df['item'].apply(lambda x: x.replace('% :', ''))
final_df.head()

,user,item,rating
0,55,إيقاظ علمي (الحواس),NaN
1,71,إيقاظ علمي (الحواس),NaN
2,114,إيقاظ علمي (الحواس),NaN
3,118,إيقاظ علمي (الحواس),NaN
4,119,إيقاظ علمي (الحواس),NaN


In [34]:
def custom_recommander1(final_df, subject, user_id):

    if user_id not in list(final_df['user']):
      return("User ID not found")
    # select sub-dataset based on the module 
    selected = [i  for i in set(final_df["item"]) if subject in i]
    sub = final_df.loc[final_df['item'].isin(selected)] 
    sub = sub.dropna()

    # Specifiy the reader scale
    reader = Reader(rating_scale=(0, 10))
    
    # Loads Pandas dataframe
    data = Dataset.load_from_df(sub[["user", "item", "rating"]], reader)
    trainingSet = data.build_full_trainset()

    # To use item-based cosine similarity
    sim_options = {
        "name": "cosine",
        "user_based": False,
        "similarity_based" : True  # Compute  similarities between items
    }
    # Build the algo training 
    algo = KNNWithMeans(sim_options=sim_options, random_state=0)
    # Fit the model
    algo.fit(trainingSet)
    
    # List the recommandation for the selected user 
    recommandation = {"module" :[], "actual progress": [], "recommanded" : []}
    for module in set(sub['item']) :
        prediction = algo.predict(user_id, module)
        recommandation["module"].append(module)
        recommandation["actual progress"].append(((final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"]).iloc[0])
        recommandation["recommanded"].append(prediction.est)
        #print(module , ':', prediction.est, (final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"])
    rec = pd.DataFrame(recommandation)  
    rec = rec.fillna(0)
    rec = rec.sort_values(by=['recommanded',"actual progress"])  
    return(rec)

In [35]:
recommandations = custom_recommander1(final_df, "رياضيات", 71388)
try :
   print('Recommandation Results:')
   print(" * Top recommanded module :",str(recommandations.iloc[-1]['module']))
   print(" * Least recommanded module :",str(recommandations.iloc[0]['module']))
except :
   pass
recommandations

Computing the cosine similarity matrix...
Done computing similarity matrix.
Recommandation Results:
 * Top recommanded module : رياضيات (الضرب في 3 أرقام)
 * Least recommanded module : رياضيات (حساب ذهني)


,module,actual progress,recommanded
5,رياضيات (حساب ذهني),0.00,0.000000
8,رياضيات (أتصرف في القطع النقدية و الأوراق الما...,0.00,0.000000
10,رياضيات (ألعاب تحدي درجة 3),0.00,0.000000
12,رياضيات ((الأعداد ذات 6 أرقام (أفكك / أركب / ...,0.00,0.000000
15,رياضيات (الضرب في عقد أو مائة),0.00,0.000000
23,"رياضيات (أتصرف في وحدات قيس الأطوال , المتر و ...",0.00,0.000000
0,رياضيات (أتصرّف في وحدات قيس الكتل),0.00,0.004818
9,"رياضيات (الأعداد ذات خمسة أرقام , مقارنة و ترتيب)",0.00,0.009540
13,"رياضيات (المسالك على الشبكة , اختصار المسالك و...",0.00,0.026392
3,رياضيات (أحسب قيس محيط شكل),0.00,0.027485


In [36]:
def custom_recommander2(final_df, subject, user_id):

    if user_id not in list(final_df['user']):
      return("User ID not found")
    # select sub-dataset based on the module 
    selected = [i  for i in set(final_df["item"]) if subject in i]
    sub = final_df.loc[final_df['item'].isin(selected)] 
    sub = sub.dropna()

    # Specifiy the reader scale
    reader = Reader(rating_scale=(0, 10))

    # Loads Pandas dataframe
    data = Dataset.load_from_df(sub[["user", "item", "rating"]], reader)
    trainingSet = data.build_full_trainset()

    
    """param_grid = {
    "n_epochs": [5, 10],
    "lr_all": [0.002, 0.005],
    "reg_all": [0.4, 0.6]
      }"""
      
    # Build the algo training 
    algo = SVD(n_epochs = 15, lr_all=0.001, reg_all=0.2, random_state=0)
    # Fit the model
    algo.fit(trainingSet)
    
    # List the recommandation for the selected user 
    recommandation = {"module" :[], "actual progress": [], "recommanded" : []}
    for module in set(sub['item']) :
        prediction = algo.predict(user_id, module)
        recommandation["module"].append(module)
        recommandation["actual progress"].append(((final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"]).iloc[0])
        recommandation["recommanded"].append(prediction.est)
        #print(module , ':', prediction.est, (final_df.loc[(final_df['user'] == user_id) & (final_df['item'] == module)])["rating"])
    rec = pd.DataFrame(recommandation)  
    rec = rec.fillna(0)
    rec = rec.sort_values(by=['recommanded',"actual progress"])  
    return(rec)

In [37]:
recommandations = custom_recommander2(final_df, "رياضيات", 71388)
try :
   print('Recommandation Results:')
   print(" * Top recommanded module :",str(recommandations.iloc[-1]['module']))
   print(" * Least recommanded module :",str(recommandations.iloc[0]['module']))
except :
   pass
recommandations

Recommandation Results:
 * Top recommanded module : رياضيات (أوظّف التّعامد والتّوازي في تصنيف المضلّعات)
 * Least recommanded module : رياضيات (أتصرف في القطع النقدية و الأوراق المالية)


,module,actual progress,recommanded
8,رياضيات (أتصرف في القطع النقدية و الأوراق الما...,0.00,0.221419
10,رياضيات (ألعاب تحدي درجة 3),0.00,0.222720
5,رياضيات (حساب ذهني),0.00,0.243167
3,رياضيات (أحسب قيس محيط شكل),0.00,0.282981
15,رياضيات (الضرب في عقد أو مائة),0.00,0.294278
7,رياضيات (مِساحة المستطِيلِ),0.00,0.377819
22,رياضيات (ألعاب تحدي درجة 1),0.00,0.378116
18,رياضيات (إِنجاز عمليّة قِسمة قاسِمها عدد ذو رق...,0.00,0.382259
17,رياضيات (أكون الأعداد ذات 5 أرقام و أفككها و أ...,0.08,0.388029
23,"رياضيات (أتصرف في وحدات قيس الأطوال , المتر و ...",0.00,0.403083
